#컨텐츠 기반 필터링
> 컨텐츠의 속성 정보를 이용하여 유사성을 판단하는 기법이다. 예를 들어 영화 추천 서비스를 만든다면 영화가 가지고 있는 속성(장르, 감독, 배우, 평점, 키워드, 영화 설명 등)을 이용하여 추천 서비스를 만들 수 있다.

##1. 파일 읽기
> tmdb_5000_movies.csv 파일을 읽어 크기를 확인하세요

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

filepath = '/content/drive/MyDrive/5. 컨텐츠 기반/data/tmdb_5000_movies.csv'
df = pd.read_csv(filepath)
df.shape

(4803, 20)

> 4803개의 영화화 20의 feature가 존재하는 것을 알 수 있다.

##2. feature 줄이기

In [3]:
df.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


> 위의 feature 중 다음과 같은 내용만 추출해 보자
* id : 영화 아이디
* title : 영화 제목
* genres : 장르
* vote_average : 평점
* vote_count : 투표수
* popularity : 인기도
* keywords : 주요 키워드
* overview : 영화 개요

In [ ]:
df_movies = df[['id','title','genres','vote_average','vote_count','popularity','keywords','overview']]
df_movies

##3. 데이터 클린징
> 위의 데이터를 이용하여 이상치 및 결측치를 확인하세요

###3.1 결측치 확인

In [5]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            4803 non-null   int64  
 1   title         4803 non-null   object 
 2   genres        4803 non-null   object 
 3   vote_average  4803 non-null   float64
 4   vote_count    4803 non-null   int64  
 5   popularity    4803 non-null   float64
 6   keywords      4803 non-null   object 
 7   overview      4800 non-null   object 
dtypes: float64(2), int64(2), object(4)
memory usage: 300.3+ KB


> 전체 4803개의 데이터 중 overview가 3개의 결측치를 가지고 있다.

In [6]:
df_movies[df_movies['overview'].isnull()]

,id,title,genres,vote_average,vote_count,popularity,keywords,overview
2656,370980,Chiamatemi Francesco - Il Papa della gente,"[{""id"": 18, ""name"": ""Drama""}]",7.3,12,0.738646,"[{""id"": 717, ""name"": ""pope""}, {""id"": 5565, ""na...",NaN
4140,459488,"To Be Frank, Sinatra at 100","[{""id"": 99, ""name"": ""Documentary""}]",0.0,0,0.050625,"[{""id"": 6027, ""name"": ""music""}, {""id"": 225822,...",NaN
4431,292539,Food Chains,"[{""id"": 99, ""name"": ""Documentary""}]",7.4,8,0.795698,[],NaN


> overview에 3개의 결측치가 존재하는 것을 확인했다. 우리가 분석할 내용에서 overview는 별도로 분석하지 않을 것이므로 무시할 것이다.

###3.2 이상치 확인

In [7]:
df_movies.describe()

,id,vote_average,vote_count,popularity
count,4803.000000,4803.000000,4803.000000,4803.000000
mean,57165.484281,6.092172,690.217989,21.492301
std,88694.614033,1.194612,1234.585891,31.816650
min,5.000000,0.000000,0.000000,0.000000
25%,9014.500000,5.600000,54.000000,4.668070
50%,14629.000000,6.200000,235.000000,12.921594
75%,58610.500000,6.800000,737.000000,28.313505
max,459488.000000,10.000000,13752.000000,875.581305


##4. 데이터 확인(팁)

In [8]:
df_movies[['genres', 'keywords']][:3]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na..."
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name..."


> 전체 데이터가 화면에 다 표시되지 않는다.

In [ ]:
pd.set_option('max_colwidth', 100)
df_movies[['genres', 'keywords']][:3]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""name"": ""Action""}]","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""name"": ""drug abuse""}, {""id"": 911, ""name"": ""exotic is..."
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 80, ""name"": ""Crime""}]","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name"": ""based on novel""}, {""id"": 4289, ""name"": ""secret..."


>pandas의 set_option을 이용하여 colunm의 최대 길이를 증가 시킬 수 있다.

In [ ]:
pd.reset_option('max_colwidth')
df_movies[['genres', 'keywords']][:3]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na..."
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name..."


> reset_option을 이용하면 초기화 시킬 수 있다.

##5. 데이터 가공

In [9]:
str_dict = '{"id": 28, "name": "Action"}'
print(type(str_dict))
print(str_dict)

<class 'str'>
{"id": 28, "name": "Action"}


>type은 str 이며 데이터는 dict 형식을 갖게 된다.

In [10]:
str_dict['id']

TypeError: ignored

> 위의 코드에서 dict 모양으로 되어 있어 id부분을 접근해 보았더니 에러가 발생한다. 이는 모양만 dict인 문자열이기 때문에 접근이 불가능하게 되는 것이다.

In [11]:
from ast import literal_eval

dict = literal_eval(str_dict)

print(type(dict))
print(dict)
print(dict['id'])

<class 'dict'>
{'id': 28, 'name': 'Action'}
28


> 문자열을 dict 형으로 변경하기 위해 literal_eval 함수를 이용했다.

In [17]:
#모든 데이터 출력
# df_movies['genres']
#1행만 출력, series 형식
# print(df_movies['genres'][:1])
#1행만 출력, str 형식
# print(df_movies['genres'][:1][0])
df_movies['genres'][:1][0]
#문자열에 0번 위치의 값
# df_movies['genres'][:1][0][0]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

>"df_movies['genres'][:1][0]" 이 정보는 리스트 모양을 갖지만 실제 자료형은 str 형식이다. 리스트에서 첫 번째 데이터인 {"id": 28, "name": "Action"} 이 정보를 얻으려 df_movies['genres'][:1][0][0] 이렇게 진행했지만 결과는 str 형식이기 때문에 첫 번째 위치의 '['이 출력된 것이다.

In [ ]:
df_movies['genres'] = df_movies['genres'].apply(literal_eval)
print(type(df_movies['genres'][:1][0]))
df_movies['genres'][:1][0][0]

> literal_eval 함수를 이용하여 str 형식을 list형식으로 변환시키고 df_movies['genres'][:1][0][0]을 이용하여 첫 번째 데이터를 읽어 들였더니 원하는 {'id': 28, 'name': 'Action'} 정보가 출력되었다. 이렇게 정상적인 데이터 가공을 위해 문자열 형식을 기존의 자료형으로 변환하는 작업이 필요하다

In [20]:
df_movies['genres'][0]

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

> 장르 첫보의 첫 데이터를 확인하면 총 4개의 장르 정보가 속해 있는 것을 볼 수 있다.

In [21]:
for genre in df_movies['genres'][0]:
  print(genre['name'])

Action
Adventure
Fantasy
Science Fiction


> 위와 같이 장르 정보만 별도로 추출하기 위해 dict 정보의 name 정보만 출력했다.

In [22]:
df_movies['genres'] = df_movies['genres'].apply(lambda x : [y['name'] for y in x])
df_movies['genres'][2]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


['Action', 'Adventure', 'Crime']

> apply를 이용하여 장르 부분에 기존 데이터 대신 name 정보만 입력했다.

##6. feature 생성

In [ ]:
'==='.join(df_movies['genres'][0])

'Action===Adventure===Fantasy===Science Fiction'

>join은 각 데이터에 구분자를 지정하는 함수이다. 위에서 0번째 위치의 데이터를 '==='의 식별자로 구분하여 결과를 출력했다.

In [23]:
df_movies['genres_literal'] = df_movies['genres'].apply(lambda x : (' ').join(x))
df_movies['genres_literal'][0]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


'Action Adventure Fantasy Science Fiction'

> CountVectorizer에서 사용하기 위해 데이터를 띄어쓰기를 구분자로 저장

In [24]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(ngram_range=(1, 2)).fit(df_movies['genres_literal'])
count_vect.vocabulary_

{'action': 0,
 'action adventure': 1,
 'action animation': 2,
 'action comedy': 3,
 'action crime': 4,
 'action drama': 5,
 'action family': 6,
 'action fantasy': 7,
 'action history': 8,
 'action horror': 9,
 'action mystery': 10,
 'action romance': 11,
 'action science': 12,
 'action thriller': 13,
 'action war': 14,
 'action western': 15,
 'adventure': 16,
 'adventure action': 17,
 'adventure animation': 18,
 'adventure comedy': 19,
 'adventure crime': 20,
 'adventure documentary': 21,
 'adventure drama': 22,
 'adventure family': 23,
 'adventure fantasy': 24,
 'adventure history': 25,
 'adventure horror': 26,
 'adventure mystery': 27,
 'adventure romance': 28,
 'adventure science': 29,
 'adventure thriller': 30,
 'adventure war': 31,
 'adventure western': 32,
 'animation': 33,
 'animation action': 34,
 'animation adventure': 35,
 'animation comedy': 36,
 'animation drama': 37,
 'animation family': 38,
 'animation fantasy': 39,
 'animation music': 40,
 'animation mystery': 41,
 'anim

> CountVectorizer에서 ngram_range는 단어의 조합을 어떻게 처리할 것이냐를 정하는 것으로 다음과 같이 사용할 수 있다.
* ngram_range(min_n, max_n)
* min_n : 최소 단어 수
* max_n : 최대 단어수
* 최소 단어수에서 최대 단어 수까지 조합으로 count할 단어를 생성

> 단어가 너무 많으면 처리하는데 많은 시간이 소요되어 2개의 단어조합 까지만 사용한다.

In [25]:
count_vect = CountVectorizer(min_df=0, ngram_range=(1, 2)).fit(df_movies['genres_literal'])
count_vect.vocabulary_

{'action': 0,
 'action adventure': 1,
 'action animation': 2,
 'action comedy': 3,
 'action crime': 4,
 'action drama': 5,
 'action family': 6,
 'action fantasy': 7,
 'action history': 8,
 'action horror': 9,
 'action mystery': 10,
 'action romance': 11,
 'action science': 12,
 'action thriller': 13,
 'action war': 14,
 'action western': 15,
 'adventure': 16,
 'adventure action': 17,
 'adventure animation': 18,
 'adventure comedy': 19,
 'adventure crime': 20,
 'adventure documentary': 21,
 'adventure drama': 22,
 'adventure family': 23,
 'adventure fantasy': 24,
 'adventure history': 25,
 'adventure horror': 26,
 'adventure mystery': 27,
 'adventure romance': 28,
 'adventure science': 29,
 'adventure thriller': 30,
 'adventure war': 31,
 'adventure western': 32,
 'animation': 33,
 'animation action': 34,
 'animation adventure': 35,
 'animation comedy': 36,
 'animation drama': 37,
 'animation family': 38,
 'animation fantasy': 39,
 'animation music': 40,
 'animation mystery': 41,
 'anim

> min_df는 최소 빈도 수를 의미하며 0을 사용하면 모든 단어들의 조합을 사용하겠다는 의미이다. 만약 100이 입력되면 100번 이하로 나타나는 조합은 배제하겠다는 의미이다.

In [26]:
genre_matrix = count_vect.fit_transform(df_movies['genres_literal'])
genre_matrix.shape

(4803, 276)

> 장르를 총 276개의 feature로 구분하여 나열했다.

##7. 유사도 분석

In [27]:
from sklearn.metrics.pairwise import cosine_similarity

genre_similarity = cosine_similarity(genre_matrix, genre_matrix)
genre_similarity[:3]

array([[1.        , 0.59628479, 0.4472136 , ..., 0.        , 0.        ,
        0.        ],
       [0.59628479, 1.        , 0.4       , ..., 0.        , 0.        ,
        0.        ],
       [0.4472136 , 0.4       , 1.        , ..., 0.        , 0.        ,
        0.        ]])

>간단하게 유사도 측정을 했다. 총 4803개의 데이터가 존재하며 각 데이터들의 유사도로 나타나기 때문에 영화를 찾기에는 어렵다. 따라서 결과를 정렬된 index 정보로 출력해 볼 것이다.

In [28]:
genre_similarity_sorted_idx = genre_similarity.argsort()[:,::-1]
genre_similarity_sorted_idx[:3]

array([[   0, 3494,  813, ..., 3038, 3037, 2401],
       [ 262,    1,  129, ..., 3069, 3067, 2401],
       [   2, 1740, 1542, ..., 3000, 2999, 2401]])

> argsort()는 배열 정렬시 사용하는 함수로 기본은 오름차순으로 정렬된다. 하지만 우리가 필요한 정보는 유사도가 높은 정보순으로 정렬해야 함으로 내림차순으로 정렬하기 위해 [:,::-1]을 이용했다. 행은 모든 값을 사용하며 열에 대해 내림차순으로 정렬한다. 또한 출력 결과는 유사도가 아닌 index인 것에 유의하자.

> 위의 정보에서 0번 영화와 가장 유사한 영화는 3493->813 순 인것을 알 수있다.

In [29]:
print(df_movies.iloc[0]['title'])
print(df_movies.iloc[3493]['title'])
print(df_movies.iloc[813]['title'])
print('==============================')
df_movies.iloc[0]


Avatar
Morvern Callar
Superman


id                                                            19995
title                                                        Avatar
genres                [Action, Adventure, Fantasy, Science Fiction]
vote_average                                                    7.2
vote_count                                                    11800
popularity                                                  150.438
keywords          [{"id": 1463, "name": "culture clash"}, {"id":...
overview          In the 22nd century, a paraplegic Marine is di...
genres_literal             Action Adventure Fantasy Science Fiction
Name: 0, dtype: object

>iloc를 이용하면 index의 정보를 얻을 수 있다. 즉, 영화의 정보를 얻기 위해 iloc에 index 정보를 넣고 그 중 title feature를 출력하여 추천 영화들을 얻을 수 있게 된다.

##8. 영화 추천하기
> 제목을 입력하면 특정 영화 정보가 출력되도록 코딩하시오

In [ ]:
# df_movies['title']
# df_movies['title']=='Avatar'
df_movies[df_movies['title']=='Avatar']

In [ ]:
# df_movies['title'].isin(['Avatar'])
df_movies[df_movies['title'].isin(['Avatar'])]

> 위와 같이 비교 연산을 이용해도 되고 isin 함수를 이용하여 데이터를 얻을 수도 있다.

In [ ]:
movie_name = 'avatar'
movie_name.upper()
movie_name.lower()
df_movies[df_movies['title'].str.lower().isin([movie_name.lower()])]

> 사용자가 입력하는 정보는 대문자, 소문자를 구분하지 않고 사용하기 때문에 하나로 통일해야 검색이 가능하다. upper 함수는 모든 문자를 대문자로 변환하며 lower 함수는 모든 문자를 소문자로 변환한다. 

> 여기서는 사용자가 입력하는 문자를 소문자로 변환하기 위해 movie_name.upper()를 사용했다. 또한 기존에 영화 제목 정보들은 series 형으로 되어 있어 문자열로 변환하기 위해 str을 사용했으며 마찬가지로 lower함수를 통해 소문자로 통일하였다. 

In [44]:
movie_name='superman'
title_movie = df_movies[df_movies['title'].str.lower().isin([movie_name.lower()])]
title_movie.index.values

array([813])

> 찾고자 하는 영화의 제목을 입력하면 영화 정보를 얻었다. 하지만 우리가 하고 싶은 것은 입력한 영화에 대한 추천 영화를 받고 싶기 때문에 index 정보가 필요하다. 그래서 검색한 영화 정보에서 index.values를 이용하여 찾고자 하는 영화의 index 정보를 얻게 되었다.

In [ ]:
genre_similarity_sorted_idx[:4, :10]

> [:4, :10]은 행을 4개까지 열을 10개까지 출력하는 기능을 한다.

> 위의 결과를 보면 4개의 영화 정보에 대한 10개의 추천 영화 index를 반환한 결과이다.

In [46]:
genre_similarity_sorted_idx[0, :10]

array([   0, 3494,  813,  870,   46,   14, 1296, 1652,  419,  420])

> 위의 결과는 0번 영화에 대한 top10의 추천 영화정보이다.

In [48]:
movie_index = title_movie.index.values
movie_index
genre_similarity_sorted_idx[movie_index, :10]

array([[   0, 3494,  813,  870,   46,   14, 1296, 1652,  419,  420]])

> 사용자가 입력한 정보에 대한 상위 10개의 추천 영화 index 정보이다.

In [ ]:
df_movies.iloc[[   0, 3494,  813,  870,   46,   14, 1296, 1652,  419,  420]]

> index 정보를 영화 정보로 얻을 수 있다.

###8.1 Quiz
> 영화 이름을 검색하면 10개의 추천 영화 제목과 평점을 출력하도록 코딩하시오
* 힌트 : reshape 함수를 활용하세요

In [ ]:
#영화 이름을 입력 받으면 index 정보 찾기
movie_name = 'avatar'
title_movie = df_movies[df_movies['title'].str.lower().isin([movie_name.lower()])]
title_movie

movie_index = title_movie.index.values
movie_index
#index정보를 이용하여 top10
movie_top10 = genre_similarity_sorted_idx[movie_index, :10]
print(type(movie_top10))

print(movie_top10.ndim, '차원 : ', movie_top10)
movie_top10 = movie_top10.reshape(-1)
print(movie_top10.ndim, '차원 : ', movie_top10)
#top10정보를 이용하여 영화 제목, 평점 추출
df_movies.iloc[movie_top10]
# df_movies.iloc[[   0, 3494,  813,  870,   46,   14, 1296, 1652,  419,  420]]

> ndim은 배열의 차원을 나타내는 객체이다. 현재 movie_top10의 차원을 알아 보기 위해 ndim을 사용하였다.

> reshape 함수는 차원을 변경하기 위한 함수로 -1을 입력하게 되면 그 차원은 자동으로 설정하게 되며 현재는 -1만 입력되어 1차원으로 설정되게 된다.

> top 10의 정보가 이차원 리스트 정보로 나타남으로 iloc에서 에러가 발생된다. 이를 해결하기 위해 reshape함수를 이용하여 일차원 리스트로 변경하고 top10의 값을 추출할 수 있다.

In [ ]:
df_movie_top10 = df_movies.iloc[movie_top10]
df_movie_top10[['title', 'vote_average']]

> 위와 같이 입력된 영화에 따른 top10의 영화를 추천해 줄 수 있다.

##9. 가중치

In [83]:
df_movies[['title', 'vote_average', 'vote_count']].sort_values('vote_average', ascending=False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


>위에서 보듯이 평점이 좋지만 투표수가 적은 경우의 영화를 추천하기에는 문제가 있다. 따라서 가중치를 이용한 추천 시스템을 구축할 것이다. 다음은 유명한 영화 평점 사이트인 IMDB에서 사용한 공식이다.
* 가중 평점 = (v/(v+m)) * R + (m/(v+m)) * C
* v : 개별 영화에 평점을 투표한 횟수
* m : 평점을 부여하기 위한 최소 투표 횟수
* R : 개별 영화에 대한 평균 평점
* C : 전체 영화에 대한 평균 평점

In [77]:
#전체 영화에 대한 평균 평점
C = df_movies['vote_average'].mean()
#평점을 부여하기 위한 최소 투표 횟수
m = df_movies['vote_count'].quantile(0.6)
print('C : ', round(C, 3))
print('m : ', round(m, 3))

C :  6.092
m :  370.2


> 평균 평점을 구하기 위해 mean 함수를 이용했다. quantile 함수는 입력된 데이터의 기준 값을 얻는 함수이다. quantile(0.6)은 60% 지점의 값을 얻은 것이며 370회의 투표를 한 것임을 알 수 있다.

In [ ]:
# 개별 영화에 평점을 투표한 횟수
v = df_movies['vote_count']
#개별 영화에 대한 평균 평점
R = df_movies['vote_average']

(v/(v+m)) * R + (m/(v+m)) * C

> 공식을 이용하여 각 영화들의 가중치 평점을 구했다.

In [ ]:
df_movies['weighted_vote'] = (v/(v+m)) * R + (m/(v+m)) * C
df_movies[['title', 'vote_average','weighted_vote', 'vote_count']].sort_values('vote_average', ascending=False)[:10]

> 실행 결과 투표수가 적은 영화는 가중치 평점이 낮았으며 투표수가 많은 영화일 수록 실 평점과 비슷한 수준으로 가중치 평점이 나타났다. 이를 이용하여 영화를 추천해 보자

###9.1 Quiz
> 가중치 평균을 이용하여 상위 10개의 영화를 추천하는 프로그램을 만드시오

In [ ]:
#영화 이름을 입력 받으면 index 정보 찾기
movie_name = 'avatar'
title_movie = df_movies[df_movies['title'].str.lower().isin([movie_name.lower()])]
title_movie

movie_index = title_movie.index.values
movie_index
#index정보를 이용하여 top10
movie_top10 = genre_similarity_sorted_idx[movie_index, :20]
movie_top10 = movie_top10.reshape(-1)

> 기존 코드에서 상위 20개의 추천 영화를 추출했다.

In [ ]:
# 추천 영화 중 검색한 영화 제외
movie_top10 = movie_top10[movie_top10!=movie_index]
movie_top10

> 추천 영화 중 검색한 영화를 제외하기 위해 != 연산을 처리했다.

In [ ]:
#top10정보를 이용하여 영화 제목, 평점 추출
df_movie_top10 = df_movies.iloc[movie_top10].sort_values('weighted_vote', ascending=False)[:10]
df_movie_top10[['title', 'vote_average','weighted_vote', 'vote_count']]

> 가중치 평점 기준 내림차순으로 상위 10개의 영화를 추천